In [ ]:
###Імпортуємо необхідні бібліотеки для роботи з aws, викликавши сабпроцес, що виконує bash скрипт по встановленню саме необхідних бібліотек.

In [1]:
import sys
import subprocess

exit_code = subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
if exit_code:
    raise Exception("Non-zero return code when installing the dependencies, please try reinstalling them manually")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
##Імпортуємо бібліотеки boto3 та requests. Створюємо секьюріті группу. Визначаємо свій ip, з якого якраз і буде доступ по http та ssh на наш інстанс

In [27]:
import boto3
import requests

group_name = "packer-sec-group"
ec2_client = boto3.client("ec2", "eu-central-1")

my_cidr = f'{requests.get("https://api.ipify.org?format=json").json().get("ip", "127.0.0.1")}/32'
print(my_cidr)

output = ec2_client.create_security_group(
    Description="Security Group for packer",
    GroupName=group_name,
    VpcId="vpc-0f0802b0c6dd27cb8"
)

group_id = output.get('GroupId')
print("Successfully created security group")

ec2_client.authorize_security_group_ingress(
    GroupId=group_id,
    IpPermissions=[{
        "IpProtocol": "tcp",
        "FromPort": 80,
        "ToPort": 80,
        "IpRanges": [{
            "CidrIp": my_cidr,
            "Description": "HTTP from my laptop"
        }]
    }, {
        "IpProtocol": "tcp",
        "FromPort": 22,
        "ToPort": 22,
        "IpRanges": [{
            "CidrIp": my_cidr,
            "Description": "SSH from my laptop"
        }]
    }]
)

46.98.5.120/32
Successfully created security group


{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0178b88fd8fdf2681',
   'GroupId': 'sg-03782364e77fac8c3',
   'GroupOwnerId': '401952100199',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 80,
   'ToPort': 80,
   'CidrIpv4': '46.98.5.120/32',
   'Description': 'HTTP from my laptop'},
  {'SecurityGroupRuleId': 'sgr-0208c102dcfb3ed68',
   'GroupId': 'sg-03782364e77fac8c3',
   'GroupOwnerId': '401952100199',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '46.98.5.120/32',
   'Description': 'SSH from my laptop'}],
 'ResponseMetadata': {'RequestId': '35daa868-4541-429b-82db-39610c9f135e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '35daa868-4541-429b-82db-39610c9f135e',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '1251',
   'date': 'Tue, 02 May 2023 11:29

In [ ]:
###Знаючи ami id, запускаємо інстанс. Тут має бути параметр IamInstanceProfile, але в моєму акаунті не має IAM Role. Так і не розібралась, як правильно створити. User data docker run прибрала, т.як в моїй амішці docker restart always, тобто сам підіймається при старті машинки.

In [31]:
import boto3

ami_id = "ami-0ec6c5012745bfce9"

ec2_client = boto3.client("ec2", "eu-central-1")

output = ec2_client.run_instances(
  MaxCount=1,
  MinCount=1,
  InstanceType="t3.micro",
  ImageId=ami_id,
  KeyName="geeshka",
  SecurityGroupIds=[
    group_id,
  ],
)

instance_ids = list(map(lambda i : i.get("InstanceId"), output.get("Instances")))
print(f"Launched instances with ids: {' '.join(instance_ids)}")

jdun = ec2_client.get_waiter("instance_running")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are up and running")

Launched instances with ids: i-084f6620cb98f8c2d
Instances with ids: i-084f6620cb98f8c2d are up and running


In [ ]:
###Дізнаємось публічну адресу інстанса, щоб до нього підключитись по ssh

In [32]:
ec2_resource = boto3.resource("ec2", "eu-central-1")
instance = ec2_resource.Instance(instance_ids[0])
print(instance.public_ip_address)

3.77.37.219


In [ ]:
###Видаляємо машинку

In [33]:
ec2_client.terminate_instances(InstanceIds=instance_ids)
jdun = ec2_client.get_waiter("instance_terminated")
jdun.wait(InstanceIds=instance_ids)
print(f"Instances with ids: {' '.join(instance_ids)} are terminated")

Instances with ids: i-084f6620cb98f8c2d are terminated
